In [1]:
from dotenv import load_dotenv
from crewai import Crew
from tasks import MemoryTasks
from agents import LTMAgents
import os
load_dotenv()


CLEARING MEMORY


True

In [2]:
user_context = "I want to create a meal plan for myself for lunch but Im allergic to fish. Im also a vegetarian?"
subsequent_request = 'can you provide a meal plan for me?'

In [3]:
os.environ['OPENAI_API_BASE']='https://api.groq.com/openai/v1'
os.environ['OPENAI_MODEL_NAME']='llama3-70b-8192'  # Adjust based on available model
os.environ['OPENAI_API_KEY']=os.environ["GROQ_API_KEY"] 

In [4]:

tasks = MemoryTasks()
agents = LTMAgents()

# create agents
context_agent = agents.context_agent()
retrieval_agent = agents.retrieval_agent()
user_proxy_agent = agents.user_proxy_agent()

# create tasks
provide_nutrition_task = tasks.provide_nutrition_task(user_proxy_agent, user_context)
analyze_and_insert_task = tasks.analyze_and_insert_task(context_agent, user_context)
analyze_and_insert_meaningful_info_task = tasks.analyze_and_insert_meaningful_info_task(context_agent, user_context)
retrieve_and_combine_relevant_memories_task = tasks.retrieve_and_combine_relevant_memories_task(retrieval_agent, subsequent_request)
provide_nutrition_task2 = tasks.provide_nutrition_task(user_proxy_agent, subsequent_request)

provide_nutrition_task2.context = [retrieve_and_combine_relevant_memories_task]

    
crew = Crew(
    agents=[
        user_proxy_agent,
        context_agent,
        retrieval_agent
    ],
    tasks=[
        provide_nutrition_task,
        analyze_and_insert_task,
        analyze_and_insert_meaningful_info_task,
        retrieve_and_combine_relevant_memories_task,
        provide_nutrition_task2
    ]
)

In [5]:
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 

I'd be happy to help you create a personalized meal plan for lunch that accommodates your vegetarian diet and fish allergy! 

Firstly, let's focus on the essential nutrients that you need to include in your meal plan to ensure you're getting a balanced diet. As a vegetarian, you'll want to prioritize plant-based sources of protein, healthy fats, and complex carbohydrates. 

Here's a 7-day lunch meal plan tailored to your requirements:

**Monday**
Lunch: Hummus and Avocado Wrap
* Whole wheat wrap filled with hummus, sliced avocado, cucumber, bell peppers, and mixed greens
* Side salad with cherry tomatoes, carrots, and a citrus vinaigrette dressing

**Tuesday**
Lunch: Quinoa and Black Bean Bowl
* Cooked quinoa mixed with black beans, roasted sweet potatoes, diced tomatoes, and chopped cilantro
* Side of steamed broccoli with a squeeze of lemon juice

**Wednesday**
Lunch: Grilled Veggie Sandwich
*

In [6]:
print(result)

Dear valued user,

I'm delighted to provide you with a personalized meal plan, carefully crafted to cater to your vegetarian lifestyle and fish allergy. Here's a 7-day meal plan, tailored to ensure you receive a balanced and nutritious diet while avoiding any potential allergens.

**Monday**

* Breakfast: Whole-grain toast with scrambled tofu and sautéed spinach, cherry tomatoes, and avocado
* Lunch: Quinoa salad with roasted vegetables (bell peppers, zucchini, and cherry tomatoes), mixed greens, and a citrus vinaigrette dressing
* Dinner: Lentil soup with a side of steamed broccoli and whole-grain bread

**Tuesday**

* Breakfast: Smoothie bowl with almond milk, banana, spinach, and almond butter topping
* Lunch: Grilled portobello mushroom burgers on whole-grain buns with roasted red peppers and sweet potato fries
* Dinner: Vegetable stir-fry (broccoli, carrots, and bell peppers) with brown rice and a side of steamed green beans

**Wednesday**

* Breakfast: Overnight oats with almond 

In [7]:
from memo_store import MemoStore

store = MemoStore(verbosity=0, reset=False, path_to_db_dir="./tmp/teachable_agent_db")
store.list_memos()


LOADING MEMORY FROM DISK
    Location = ./tmp/teachable_agent_db/uid_text_dict.pkl
LIST OF MEMOS
  ID: 1
    INPUT TEXT: Create a personalized meal plan
    OUTPUT TEXT: Consider dietary restrictions and preferences, such as vegetarianism and fish allergy
  ID: 2
    INPUT TEXT: What are my dietary restrictions and preferences for a lunch meal plan?
    OUTPUT TEXT: I am a vegetarian and allergic to fish.
